In [65]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [66]:
def model_maker(IMG_WIDTH, IMG_HEIGHT, NUM_CLASSES):
    base_model = tf.keras.applications.mobilenet.MobileNet(include_top=False, input_shape =
    (IMG_WIDTH,IMG_HEIGHT,3))
    for layer in base_model.layers[:-5]:
        layer.trainable = False # Freeze the layers
    input = tf.keras.layers.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = tf.keras.layers.GlobalAveragePooling2D()(custom_model)
    custom_model = tf.keras.layers.Dense(64, activation='relu')(custom_model)
    custom_model = tf.keras.layers.Dropout(0.5)(custom_model)
    predictions = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(custom_model)
    return tf.keras.Model(inputs=input, outputs=predictions)

In [67]:
hotdogModel = model_maker(224,224,1)

In [68]:
# Now to prepare the data
# We will use the folders in the same directory.

In [69]:
import os
from pathlib import Path
path = Path(os.getcwd())
print(path)
# Getting all the paths for each image of clothing in the dataset
# os.walk returns the current walking directory, the subdirectories within it, and then finally the files within it.
for name, subdirs, files in os.walk(path):
    indent = len(Path(name).parts) - len(path.parts)
    print("    " * indent + Path(name).parts[-1] + os.sep)
    for index, filename in enumerate(sorted(files)):
        # If we have printed three items already, just go ahead and print and ellipsis and move on
        if index == 3:
            print("    " * (indent + 1) + "...")
            break
        print("    " * (indent + 1) + filename)

/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog
Not Hotdog/
    NotHotdogModel.ipynb
    hotdog.zip
    .ipynb_checkpoints/
        NotHotdogModel-checkpoint.ipynb
    hotdog/
        test/
            hot_dog/
                133012.jpg
                133015.jpg
                133245.jpg
                ...
            not_hot_dog/
                13023.jpg
                13080.jpg
                13719.jpg
                ...
        train/
            hot_dog/
                1000288.jpg
                1011661.jpg
                1013916.jpg
                ...
            not_hot_dog/
                100135.jpg
                100148.jpg
                100274.jpg
                ...
        seefood/
            test/
                hot_dog/
                    133012.jpg
                    133015.jpg
                    133245.jpg
                    ...
                not_hot_dog/
                    13023.jpg
                    13080.jpg
                    

In [70]:
# Function to get all the image paths and glob them into one large list and then convert the PosixPath's into
# strings
def image_paths(dirpath):
    # PosixPath.glob will get all the path names of a specific pattern and return all of them in a list
    return [str(path) for path in dirpath.glob("*.jpg")]

In [71]:
# Getting the paths to all the images
x_train = image_paths(path/"hotdog"/"train"/"hot_dog")
y_train = image_paths(path/"hotdog"/"train"/"not_hot_dog")
x_test = image_paths(path/"hotdog"/"test"/"hot_dog")
y_test = image_paths(path/"hotdog"/"test"/"not_hot_dog")

In [72]:
# Sanity Check
y_test[:5]

['/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/test/not_hot_dog/30892.jpg',
 '/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/test/not_hot_dog/38615.jpg',
 '/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/test/not_hot_dog/26078.jpg',
 '/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/test/not_hot_dog/51368.jpg',
 '/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/test/not_hot_dog/8350.jpg']

In [73]:
def label_image(filename):
    # Getting the last file instead of the entire file path
    depth_of_file = filename.count("/")
    filepath = filename.split("/", depth_of_file - 3)[-1]
    label = filepath.split("/", 4)[2]
    if(label == "hot_dog"):
        label = 1
    else:
        label = 0
    
    # Returning the image and the encoded label
    return filepath, label

# For showing the image
def show(image, label):
    plt.figure()
    plt.imshow(image)
    plt.title(label)
    plt.axis('off')

In [74]:
import random
train = x_train + y_train
test = x_test + y_test

In [75]:
random.shuffle(train)
random.shuffle(test)

In [76]:
# Sanity Check
print(train)

['/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/train/hot_dog/3695334.jpg', '/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/train/not_hot_dog/129045.jpg', '/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/train/not_hot_dog/105375.jpg', '/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/train/hot_dog/893274.jpg', '/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/train/not_hot_dog/105647.jpg', '/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/train/hot_dog/1211682.jpg', '/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/train/hot_dog/3911675.jpg', '/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/train/hot_dog/3746979.jpg', '/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/train/hot_dog/1313978.jpg', '/home/ryuparish/Code/machine_learning/Experiments/Not Hotdog/hotdog/train/hot_dog/958900.jpg', '/home/ryuparish/Code/

In [77]:
# This is actually not x and y data but positive and negative examples of both datasets
train = map(label_image, train)
train = list(train)
test = map(label_image, test)
test = list(test)

In [78]:
# Sanity Check
test[:5]

[('hotdog/test/hot_dog/285214.jpg', 1),
 ('hotdog/test/not_hot_dog/30731.jpg', 0),
 ('hotdog/test/hot_dog/397071.jpg', 1),
 ('hotdog/test/not_hot_dog/19487.jpg', 0),
 ('hotdog/test/not_hot_dog/46246.jpg', 0)]

In [79]:
# Dividing up the training set into validation and training data and making the test set a little smaller
train = train + test[:200]
test = test[400:]

In [80]:
valid = train[:150]
train = train[150:]

In [81]:
train = np.asarray(train)
valid = np.asarray(valid)
test = np.asarray(test)

In [82]:
x_train = train[:,0]
y_train = train[:,1].astype(np.int32).reshape((-1,1))
x_valid = valid[:,0]
y_valid = valid[:,1].astype(np.int32).reshape((-1,1))
x_test = test[:,0]
y_test = test[:,1].astype(np.int32).reshape((-1,1))

In [83]:
y_train[:5]

array([[0],
       [1],
       [1],
       [0],
       [1]], dtype=int32)

In [84]:
# Image parsing and then attaching a label based on the name of the file
# Original
from PIL import Image
def parse_image(filepath):
    # Getting the image from the filepath
    image = Image.open(filepath)
    image = image.resize((224, 224))
    image = np.asarray(image)/255
    # Returning the image and the encoded label
    return image

In [85]:
# Converting the image filepaths into their images
x_train = map(parse_image, x_train)
x_train = np.asarray(list(x_train)).astype(np.float32)
x_valid = map(parse_image, x_valid)
x_valid = np.asarray(list(x_valid)).astype(np.float32)
x_test = map(parse_image, x_test)
x_test = np.asarray(list(x_test)).astype(np.float32)

In [86]:
print(np.expand_dims(x_train, axis=0).shape)
print(x_train.shape)
print(y_train.shape)

(1, 548, 224, 224, 3)
(548, 224, 224, 3)
(548, 1)


In [87]:
print(x_valid.shape)

(150, 224, 224, 3)


In [88]:
# Converting the images with their labels into tensorflow datasets
training_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(100).batch(32).prefetch(1)
valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).shuffle(100).batch(32).prefetch(1)
testing_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1)

In [89]:
hotdogModel.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenet_1.00_224 (Function (None, 7, 7, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                65600     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 3,294,529
Trainable params: 1,118,337
Non-trainable params: 2,176,192
_________________________________________

In [92]:
hotdogModel.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=["accuracy"])

In [93]:
# This runs but it just gets the same accuracy over and over, I changed the last layer from softmax to sigmoid and it started working.
# I also changed the optimizer from nadam to adam.
hotdogModel.fit(training_dataset, epochs=10, validation_data=valid_dataset)

Epoch 1/10
18/18 [==============================] - 7s 334ms/step - loss: 0.0554 - accuracy: 0.9872 - val_loss: 3.0824 - val_accuracy: 0.7200
Epoch 2/10
18/18 [==============================] - 6s 336ms/step - loss: 0.0332 - accuracy: 0.9891 - val_loss: 1.8861 - val_accuracy: 0.7467
Epoch 3/10
18/18 [==============================] - 6s 343ms/step - loss: 0.0469 - accuracy: 0.9836 - val_loss: 1.5198 - val_accuracy: 0.8000
Epoch 4/10
18/18 [==============================] - 6s 342ms/step - loss: 0.0242 - accuracy: 0.9891 - val_loss: 2.1243 - val_accuracy: 0.7467
Epoch 5/10
18/18 [==============================] - 6s 339ms/step - loss: 0.0082 - accuracy: 0.9982 - val_loss: 1.9967 - val_accuracy: 0.7600
Epoch 6/10
18/18 [==============================] - 6s 335ms/step - loss: 0.0537 - accuracy: 0.9872 - val_loss: 1.6687 - val_accuracy: 0.8067
Epoch 7/10
18/18 [==============================] - 6s 342ms/step - loss: 0.0074 - accuracy: 0.9982 - val_loss: 1.5536 - val_accuracy: 0.8133
Epoch 

In [95]:
hotdogModel.evaluate(testing_dataset)

100/100 [==============================] - 1s 9ms/step - loss: 0.8950 - accuracy: 0.8800


[0.8950088620185852, 0.8799999952316284]

In [96]:
hotdogModel.save("HotdogModel.h5")

In [107]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope
import tensorflow.keras
with CustomObjectScope({'relu6': tensorflow.keras.layers.ReLU(6),'DepthwiseConv2D': tensorflow.keras.layers.DepthwiseConv2D}):
            model = load_model('HotdogModel.h5')
import coremltools
coreml_model = coremltools.convert(model)
#coreml_model_float32 = coremltools.convert(model, ...)

coreml_model.save('NotHotDog.mlmodel')
# Successfully converted the tf model into a coreml model

Translating MIL ==> MLModel Ops: 100%|██████████| 409/409 [00:00<00:00, 1260.31 ops/s]
